In [ ]:
#from py2neo import Graph
#graph = Graph("bolt://neo4j.sktai.io:30074", auth=("neo4j","!neo4j00"))

In [ ]:
# https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/tag/3.5.0.1
#mv ./apoc-3.5.0.1-all.jar $Neo4J_HOME/plugins

In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [2]:
#gph_conn = Neo4jConnection(uri="bolt://neo4j.sktai.io:30074", user="neo4j", pwd="!neo4j00")
#gph_conn = Neo4jConnection(uri="bolt://127.0.0.1:7687", user="neo4j", pwd="!neo4j00")
gph_conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="!neo4j00")

In [3]:
gph_conn.query("""
               CALL db.schema.visualization();
               """)

[<Record nodes=[] relationships=[]>]

In [4]:
gph_conn.query("""
          MATCH (n)
          DETACH DELETE n
          """)

[]

In [ ]:
## SET LIBRARY

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
## SET WORK PATH

In [ ]:
import os

In [ ]:
#work_path = '/home/dmig/work/this-book/neo4j-book'
work_path = '/Users/boO/data8/data.student/neo4j-book/neo4j-book'
os.chdir(work_path)

In [ ]:
tbl = pd.read_csv("{}/data/train.csv".format(work_path))

In [ ]:
tbl.shape

In [ ]:
tbl.head()

In [ ]:
node_id = np.union1d(tbl.source_node.unique(), tbl.destination_node.unique())
# len(node_id) # 1,862,220
#node_id = np.random.choice(node_id, size=100000, replace=False)
node_id = pd.DataFrame(node_id, columns=['id:ID'])
node_id[':LABEL'] = 'User'
node_id.to_csv("{}/data/node.csv".format(work_path), index=False)

In [ ]:
tbl.columns = [':START_ID',':END_ID']
tbl[':TYPE'] = 'IS_FRIENDS_WITH'
tbl.to_csv("{}/data/edge.csv".format(work_path), index=False)

In [ ]:
## LOAD DATA

In [ ]:
!bin/neo4j-admin import --nodes=import/node.csv --relationships=import/edge.csv

In [ ]:
## IT NEVER ENDS
gph_conn.query("""
               // USING PERIODIC COMMIT 500
               LOAD CSV WITH HEADERS FROM 'file:///train.csv' AS row
               MERGE (:User {id: row.source_node})
               """)
gph_conn.query("""
               LOAD CSV WITH HEADERS FROM 'file:///train.csv' AS row
               MERGE (:User {id: row.destination_node})
               """)
gph_conn.query("""
               LOAD CSV WITH HEADERS FROM 'file:///train.csv' AS row
               MATCH (s:User {id: row.source_node}) 
               MATCH (d:User {id: row.destination_node}) 
               MERGE (s)-[:IS_FRIENDS_WITH]->(d)
               """)

In [ ]:
gph_conn.query("""
               CALL db.schema();
               """)

In [ ]:
gph_conn.query("""
               PROFILE
               MATCH (u {id:"1"})
               RETURN u
               """)

In [ ]:
gph_conn.query("""
               PROFILE
               MATCH (u:User {id:"1"})
               RETURN u
               """)

In [ ]:
gph_conn.query("""
               CREATE INDEX ON :User(id)
               """)

In [ ]:
gph_conn.query("""
               PROFILE
               MATCH (u:User {id:"1"})
               RETURN u
               """)

In [ ]:
gph_conn.query("""
               MATCH (a:User {id:"3"})-[:IS_FRIENDS_WITH]->(b:User)
               RETURN COUNT(b.id)
               """)

In [ ]:
gph_conn.query("""
               EXPLAIN
               LOAD CSV WITH HEADERS FROM 'file:///train.csv' AS row
               MATCH (s:User {id: row.source_node}) 
               MATCH (d:User {id: row.destination_node}) 
               MERGE (s)-[:IS_FRIENDS_WITH]->(d)
               """)

In [ ]:
gph_conn.query("""
               CALL apoc.load.json("neo4j_repos_github.json")
               YIELD value AS item 
               RETURN item 
               LIMIT 1
               """)

In [5]:
gph_conn.query("""
               CALL apoc.load.json("neo4j_repos_github.json")
               YIELD value AS item
               CREATE (
                       r:Repository {
                                     name: item.name
               ,                     created_at: item.created_at
               ,                     contributors_url: item.contributors_url
               })
               MERGE (u:User {login: item.owner.login})
               CREATE (u)-[:OWNS]->(r)
               """)

Query failed: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `apoc.load.json` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}
